In [1]:
from scipy.stats import norm
from pandas import DataFrame
import math
import numpy as np
import pandas as pd
from enum import Enum
import requests

In [2]:
# time to maturity is in years
class Stock:
    def __init__(self, strike_price, stock_price, time_to_maturity, risk_free_rate, volatility, is_call):
        self.K = strike_price
        self.S0 = stock_price
        self.T = time_to_maturity
        self.r = risk_free_rate
        self.sigma = volatility
        self.is_call = is_call

### Black-Scholes Pricing
[Reference 1](https://www.investopedia.com/terms/b/blackscholes.asp)
[Reference 2](https://www.quantconnect.com/tutorials/introduction-to-options/options-pricing-black-scholes-merton-model)
[Reference 3](https://medium.com/swlh/calculating-option-premiums-using-the-black-scholes-model-in-python-e9ed227afbee)

In [3]:
def black_scholes(stock):
    d1 = (math.log(stock.S0 / stock.K) + (stock.r + stock.sigma ** 2 / 2) * stock.T) / (stock.sigma * math.sqrt(stock.T))
    d2 = d1 - stock.sigma * math.sqrt(stock.T)
    if stock.is_call:
        return stock.S0 * norm.cdf(d1) - norm.cdf(d2) * stock.K * math.exp(-stock.r * stock.T) 
    else:
        return stock.S0 * math.exp(-stock.r * stock.T) * norm.cdf(-d2) - stock.S0 * norm.cdf(-d1)        

### Monte Carlo Pricing (using Geometric Brownian Motion)
[Reference 1](https://towardsdatascience.com/monte-carlo-pricing-in-python-eafc29e3b6c9)
[Reference 2](https://www.quantconnect.com/tutorials/introduction-to-options/stochastic-processes-and-monte-carlo-method)
[Reference 3](https://www.codearmo.com/blog/pricing-options-monte-carlo-simulation-python)

In [4]:
def monte_carlo(stock, steps, trials):
    dt = stock.T / steps
    drift = stock.r - stock.sigma ** 2 / 2
    dWt = np.random.normal(size=(steps, trials))
    dYt = drift * dt + stock.sigma * np.sqrt(dt) * dWt
    prices = np.exp(np.log(stock.S0) + np.sum(dYt, axis=0))
    payoffs = np.maximum(prices - stock.K, 0)
    return np.average(payoffs) * np.exp(-stock.r * stock.T)

### CRR Binomial Tree Pricing
[Reference 1](https://www.linkedin.com/pulse/python-implementation-binomial-stock-option-pricing-sheikh-pancham)

In [5]:
def binomial_crr(stock, steps):
    dt = stock.T / steps
    u = np.exp(stock.sigma * np.sqrt(dt))
    d = 1 / u
    pu = (np.exp(stock.r * dt) - d) / (u - d)
    pd = 1 - pu
    
    stock_tree = np.zeros((steps + 1, steps + 1))
    prob_tree = np.zeros((steps + 1, steps + 1))
    payoff_tree = np.zeros((steps + 1, steps + 1))
    for j in range(steps + 1):
        for i in range(j + 1):
            stock_tree[i][j] = stock.S0 * u ** i * d ** (j - i)
            prob_tree[i][j] = math.factorial(j) * pu ** i * pd ** (j - i) / (math.factorial(i) * math.factorial(j - i))
    for i in range(steps + 1):
        payoff_tree[i][steps] = np.maximum(0, stock_tree[i][steps] - stock.K)
        
    payoff = 0
    for i in range(steps + 1):
        payoff += payoff_tree[i][steps] * prob_tree[i][steps]
    return payoff * np.exp(-stock.r * stock.T)

### Comparisons
Assumptions: European options, no dividend

In [6]:
stock = Stock(785, 927.96, 100/252, 0.01, 0.23, True)

print(black_scholes(stock))
print(monte_carlo(stock, 100, 1000))
print(binomial_crr(stock, 100))

153.23789489883916
151.9279472987235
153.24008036670628
